# 04 - Race Program: Model Deployment
Author: George Gorospe, george.gorospe@nmaia.net (updated 1/17/2024)

# In this fourth notebook, we'll use the the machine learning model we trained previously to pilot our vehicle around a track.

We've done a lot of work collecting images and training a model to act as the pilot of our self-driving race car. 
In this notebook, we'll input the live camera feed into the model and take steering directions as our output.s our pilot for our self-driving car.

In [ ]:
# Importing Required Libraries

# Machine Learning Libraries
import torch
from torch2trt import TRTModule
import torchvision.transforms as transforms
import torch.nn.functional as F

# Vehicle Control Libraries from invidia
from jetracer.nvidia_racecar import NvidiaRacecar
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg


# General Use Libraries
import cv2
import ipywidgets
import threading
import PIL.Image
import numpy as np


# Jupyter Labs Libraries
import traitlets
from IPython.display import display
from ipywidgets import Layout, Button, Box
import ipywidgets.widgets as widgets
from ipyfilechooser import FileChooser


# Useful Variables
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()

# Custom functions
def preprocess(image):
    device = torch.device('cuda')
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

## Starting Vehicle Control and Camera Control

In [ ]:
camera = CSICamera(width=224, height=224)
car = NvidiaRacecar()

## Selecting a trained Model
Use the file chooser to select your model

In [ ]:
# Create and display a FileChooser widget
fc = FileChooser('/home/racer_core/Models')
display(fc)
fc.filter_pattern = ['*.pth']
# Change the title (use '' to hide)
fc.title = '<b>Choose Model for Race Program </b>'

# Sample callback function
def change_title(chooser):
    chooser.title = '<b>Model Selected.</b>'

# Register callback function
fc.register_callback(change_title)

## Creating a Graphical User Interface for Race Program

In [ ]:
# Assignment for selected model
model_trt = TRTModule()
model_trt_path = fc.selected
model_trt.load_state_dict(torch.load(model_trt_path))


state_widget = ipywidgets.ToggleButtons(options=['On', 'Off'], description='Camera', value='On')
prediction_widget = ipywidgets.Image(format='jpeg', width=camera.width, height=camera.height)

live_execution_widget = ipywidgets.VBox([
    prediction_widget,
    state_widget
])

network_output_slider = widgets.FloatSlider(description='Network Output', min=-1.0, max=1.0, value=0, step=0.01, orientation='horizontal', disabled=False, layout={'width': '400px'})
steering_gain_slider  = widgets.FloatSlider(description='Steering Gain', min=-1.0, max=2.0, value=-0.7, step=0.01, orientation='horizontal', layout={'width': '300px'})
steering_bias_slider  = widgets.FloatSlider(description='Steering Bias', min=-0.5, max=0.5, value=0.0, step=0.01, orientation='horizontal', layout={'width': '300px'})
steering_value_slider = widgets.FloatSlider(description='Steering', min=-1.0, max=1.0, value=0, step=0.01, orientation='horizontal', disabled=False, layout={'width': '400px'})
throttle_slider = widgets.FloatSlider(description='Throttle', min=-1.0, max=1.0, value=0.15, step=0.01, orientation='vertical')


steering_gain_link   = traitlets.link((steering_gain_slider, 'value'), (car, 'steering_gain'))
steering_offset_link = traitlets.link((steering_bias_slider, 'value'), (car, 'steering_offset'))
#steering_value_link  = traitlets.link((steering_value_slider, 'value'), (car, 'steering'))
throttle_slider_link = traitlets.link((throttle_slider, 'value'), (car, 'throttle'))

display(
    widgets.HBox(
        [widgets.VBox([network_output_slider,
                       widgets.Label(value="X"),
                       steering_gain_slider,
                       widgets.Label(value="+"),
                       steering_bias_slider,
                       widgets.Label(value="||"), 
                       steering_value_slider], layout=Layout(
                                                    align_items='center'
                                                        )
                     ), 
         live_execution_widget,
         throttle_slider]
    )
)

def update(change):
    global blocked_slider, robot
    new_image = change['new'] 
    
    image = preprocess(new_image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    x = float(output[0])
    y = float(output[0])
    
    network_output_slider.value = x
    steering = x * steering_gain_slider.value + steering_bias_slider.value
    if(steering<-1.0):
        steering_value_slider.value = -1.0
    elif(steering>1.0):
        steering_value_slider.value = 1.0
    else:
        steering_value_slider.value = steering 
    car.steering = x
    
    if(state_widget.value == 'On'):
        x = int(camera.width * (x / 2.0 + 0.5))
        y = int(camera.height * (y / 2.0 + 0.5))  
        prediction = new_image.copy()
        prediction = cv2.circle(prediction, (x, 112), 8, (255, 0, 0), 3)
        prediction_widget.value = bgr8_to_jpeg(prediction)
        # ADD STEERING WIDGET VALUE HERE
        
update({'new': camera.value})  # we call the function once to initialize